In [ ]:
#  timetable_generator.py
# Requirements: pip install pandas numpy scikit-learn tensorflow joblib openpyxl

import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
import joblib
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers

In [ ]:
# -------------------------
# 1) Load dataset
# -------------------------
FILE_PATH = "TIMETABLE.xlsx"
MODEL_PATH = "timetable_generator.keras"
ENC_PATH = "encoders.joblib"
SCALER_PATH = "scaler.joblib"

df = pd.read_excel(FILE_PATH)
print("Columns:", df.columns.tolist())
print(df.head(3))

Columns: ['Day', 'Time Slots ', 'Subject', 'Room Number', 'Faculty']
      Day  Time Slots       Subject Room Number       Faculty
0  Monday   9:30-10:30  ECE LAB(G1)         301  SANGEETA MAM
1  Monday  10:30-11:30  ECE LAB(G1)         301  SANGEETA MAM
2  Monday  11:30-12:30          COA         301  MR.LOVEPREET


In [ ]:
# -------------------------
# 2) Clean dataset
# -------------------------
df = df.dropna(how="all")          # drop fully empty rows
df = df.fillna("Free_Slot")        # replace NaN with label

In [ ]:
# -------------------------
# 3) Decide categorical vs numeric
# -------------------------
cat_cols = []
num_cols = []

def is_number(s):
    try:
        float(s)
        return True
    except:
        return False

for c in df.columns:
    sample = df[c].astype(str).tolist()
    num_count = sum(1 for v in sample if is_number(str(v)))
    if num_count / len(sample) < 0.5:   # mostly text → categorical
        cat_cols.append(c)
    else:
        try:
            df[c] = df[c].astype(float)
            num_cols.append(c)
        except Exception:
            cat_cols.append(c)

print("Categorical cols:", cat_cols)
print("Numeric cols:", num_cols)



Categorical cols: ['Day', 'Time Slots ', 'Subject', 'Room Number', 'Faculty']
Numeric cols: []


In [ ]:
# -------------------------
# 4) Encode categorical
# -------------------------
encoders = {}
proc_df = pd.DataFrame()
for c in cat_cols:
    le = LabelEncoder()
    series = df[c].astype(str).fillna("NA")
    proc_df[c] = le.fit_transform(series)
    encoders[c] = le

# numeric as is
for c in num_cols:
    proc_df[c] = df[c].fillna(0).astype(float)

# Save encoders
joblib.dump(encoders, ENC_PATH)
print("Saved encoders ->", ENC_PATH)

Saved encoders -> encoders.joblib


In [ ]:
# -------------------------
# 5) Prepare input/output
# -------------------------
X_raw = proc_df.values.astype(np.float32)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_raw)
joblib.dump(scaler, SCALER_PATH)
print("Saved scaler ->", SCALER_PATH)

y_outputs = []
losses_dict = {}
loss_weights = {}
metrics_dict = {}
output_names = []

for c in cat_cols:
    le = encoders[c]
    n_classes = len(le.classes_)
    y = tf.keras.utils.to_categorical(proc_df[c].values, num_classes=n_classes)
    y_outputs.append(y)
    valid_name = c.replace(":", "_").replace(" ", "_").replace("\n", "_")
    losses_dict[valid_name] = "categorical_crossentropy"
    loss_weights[valid_name] = 1.0
    metrics_dict[valid_name] = ["accuracy"]
    output_names.append(valid_name)

if num_cols:
    y_num = proc_df[num_cols].values.astype(np.float32)
    y_outputs.append(y_num)
    losses_dict["numeric"] = "mse"
    loss_weights["numeric"] = 1.0
    metrics_dict["numeric"] = ["mse"]
    output_names.append("numeric")

Saved scaler -> scaler.joblib


In [ ]:
# -------------------------
# 6) Build model
# -------------------------
input_dim = X_scaled.shape[1]
latent_dim = max(8, input_dim // 2)

inp = layers.Input(shape=(input_dim,), name="encoder_input")
h = layers.Dense(latent_dim*2, activation="relu")(inp)
bottleneck = layers.Dense(latent_dim, activation="relu", name="bottleneck")(h)
d = layers.Dense(latent_dim*2, activation="relu")(bottleneck)

outputs = []
idx = 0
for c in cat_cols:
    n_classes = len(encoders[c].classes_)
    out = layers.Dense(n_classes, activation="softmax", name=output_names[idx])
    outputs.append(out(d))
    idx += 1

if num_cols:
    out_num = layers.Dense(len(num_cols), activation="linear", name="numeric")
    outputs.append(out_num(d))

model = models.Model(inputs=inp, outputs=outputs, name="timetable_generator")
model.compile(
    optimizer=optimizers.Adam(1e-3),
    loss=[losses_dict[name] for name in output_names],
    loss_weights=[loss_weights[name] for name in output_names],
    metrics=[metrics_dict[name] for name in output_names]
)
model.summary()


Model: "timetable_generator"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ encoder_input       │ (None, 5)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_11 (Dense)    │ (None, 16)        │         96 │ encoder_input[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bottleneck (Dense)  │ (None, 8)         │        136 │ dense_11[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 16)        │        144 │ bottleneck[0][0]  │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Day (Dense)         │ (None, 6)         │        102 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Time_Slots_ (Dense) │ (None, 7)         │        119 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Subject (Dense)     │ (None, 57)        │        969 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Room_Number (Dense) │ (None, 31)        │        527 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Faculty (Dense)     │ (None, 31)        │        527 │ dense_12[0][0]    │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 2,620 (10.23 KB)

 Trainable params: 2,620 (10.23 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
# -------------------------
# 7) Train
# -------------------------
history = model.fit(
    X_scaled, y_outputs,
    epochs=100,
    batch_size=32,
    validation_split=0.1,
    verbose=2
)

Epoch 1/100
15/15 - 11s - 751ms/step - Day_accuracy: 0.2163 - Day_loss: 1.7970 - Faculty_accuracy: 0.0066 - Faculty_loss: 3.4483 - Room_Number_accuracy: 0.0574 - Room_Number_loss: 3.4272 - Subject_accuracy: 0.0442 - Subject_loss: 4.0458 - Time_Slots__accuracy: 0.1457 - Time_Slots__loss: 1.9691 - loss: 14.6876 - val_Day_accuracy: 0.3137 - val_Day_loss: 1.7757 - val_Faculty_accuracy: 0.0000e+00 - val_Faculty_loss: 3.4213 - val_Room_Number_accuracy: 0.0784 - val_Room_Number_loss: 3.3654 - val_Subject_accuracy: 0.0784 - val_Subject_loss: 4.0013 - val_Time_Slots__accuracy: 0.1569 - val_Time_Slots__loss: 1.9596 - val_loss: 14.5127
Epoch 2/100
15/15 - 0s - 10ms/step - Day_accuracy: 0.2870 - Day_loss: 1.7784 - Faculty_accuracy: 0.0397 - Faculty_loss: 3.4098 - Room_Number_accuracy: 0.1060 - Room_Number_loss: 3.3981 - Subject_accuracy: 0.0640 - Subject_loss: 4.0182 - Time_Slots__accuracy: 0.1479 - Time_Slots__loss: 1.9567 - loss: 14.5597 - val_Day_accuracy: 0.3529 - val_Day_loss: 1.7698 - val_Fa

In [ ]:
# -------------------------
# 8) Save
# -------------------------
model.save(MODEL_PATH)
print("Saved model ->", MODEL_PATH)

Saved model -> timetable_generator.keras


In [ ]:
# -------------------------
# 9) Generation (sample new timetable rows)
# -------------------------
def decode_preds(preds_list, cat_cols, num_cols, encoders):
    out = {}
    p_index = 0

    for j, c in enumerate(cat_cols):
        probs = preds_list[p_index]
        idx = int(np.argmax(probs, axis=1)[0])
        out[c] = encoders[c].inverse_transform([idx])[0]
        p_index += 1
    if num_cols:
        nums = preds_list[p_index][0]
        for k, nc in enumerate(num_cols):
            out[nc] = float(nums[k])
    return out

# Example: generate 5 new rows
print("\nGenerated Timetable Rows:")
for _ in range(5):
    z = np.random.normal(size=(1, latent_dim))  # random latent vector
    noisy_input = np.random.normal(scale=0.5, size=(1, input_dim))
    preds = model.predict(noisy_input, verbose=0)
    print(decode_preds(preds, cat_cols, num_cols, encoders))


Generated Timetable Rows:
{'Day': 'Wednesday', 'Time Slots ': '2:30-3:30', 'Subject': 'Free_Slot', 'Room Number': '422', 'Faculty': 'Free_Slot'}
{'Day': 'Thursday', 'Time Slots ': '1:30-2:30', 'Subject': 'LUNCH TIME', 'Room Number': '301', 'Faculty': 'Free_Slot'}
{'Day': 'Thursday', 'Time Slots ': '2:30-3:30', 'Subject': 'Free_Slot', 'Room Number': '315', 'Faculty': 'Free_Slot'}
{'Day': 'Thursday', 'Time Slots ': '9:30-10:30', 'Subject': 'Free_Slot', 'Room Number': '401', 'Faculty': 'Free_Slot'}
{'Day': 'Monday', 'Time Slots ': '1:30-2:30', 'Subject': 'Free_Slot', 'Room Number': '301', 'Faculty': 'Free_Slot'}


# ** Genrating timetable in grid  **


In [ ]:
import pandas as pd
import random
from tabulate import tabulate

# ==============================
# Load dataset from Excel
# ==============================
INPUT_FILE = "TIMETABLE.xlsx"   # your Excel file
df = pd.read_excel(INPUT_FILE)  # columns: Subject, Faculty, Room Number

# Days and slots
days = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday"]
slots = [
    "9:30-10:30",
    "10:30-11:30",
    "11:30-12:30",
    "12:30-1:30",
    "1:30-2:30",
    "2:30-3:30",
    "3:30-4:30"
]

# ==============================
# Subject → Faculty, Room mapping
# ==============================
subject_map = {}
for _, row in df.iterrows():
    subj = str(row.get("Subject")).strip()
    fac = str(row.get("Faculty")).strip()
    room = str(row.get("Room Number")).strip()
    if subj and subj.lower() not in ["free_slot", "lunch", "nan"]:
        subject_map[subj] = (fac, room)

# ==============================
# Generate a valid row
# ==============================
def generate_row_for_subject(subject=None):
    if not subject:
        subject = random.choice(list(subject_map.keys()))
    fac, room = subject_map[subject]
    return {"Subject": subject, "Faculty": fac, "Room Number": room}

# ==============================
# Constraint: faculty cannot teach >3 continuous slots
# ==============================
def violates_faculty_streak(timetable, day, slot_idx, candidate_fac):
    count = 1
    # backward check
    for i in range(slot_idx - 1, -1, -1):
        if timetable[day].get(slots[i], "").find(candidate_fac) != -1:
            count += 1
        else:
            break
    # forward check
    for i in range(slot_idx + 1, len(slots)):
        if timetable[day].get(slots[i], "").find(candidate_fac) != -1:
            count += 1
        else:
            break
    return count > 3

# ==============================
# Constraint: prevent 2 consecutive same theory subjects
# ==============================
def violates_theory_repeat(timetable, day, slot_idx, subject, room):
    if "LAB" in subject.upper():
        return False  # labs are allowed consecutive
    # check previous slot
    if slot_idx > 0:
        prev_val = timetable[day].get(slots[slot_idx - 1], "")
        if subject in prev_val and room in prev_val:
            return True
    # check next slot
    if slot_idx < len(slots) - 1:
        next_val = timetable[day].get(slots[slot_idx + 1], "")
        if subject in next_val and room in next_val:
            return True
    return False

# ==============================
# Build timetable
# ==============================
timetable = {d: {} for d in days}

# Step 1: Assign ONE lunch break per day (randomly at 12:30–1:30 or 1:30–2:30)
for d in days:
    lunch_slot = random.choice(["12:30-1:30", "1:30-2:30"])
    timetable[d][lunch_slot] = "🍴 LUNCH BREAK"

# Step 2: Ensure each subject 2–3 times/week
required_counts = {subj: random.randint(2, 3) for subj in subject_map.keys()}
all_slots = [(d, s) for d in days for s in slots if "LUNCH" not in timetable[d].get(s, "")]
random.shuffle(all_slots)

for subj, count in required_counts.items():
    filled = 0
    while filled < count:
        placed = False
        random.shuffle(all_slots)
        for (d, s) in all_slots:
            if s in timetable[d]:
                continue
            slot_idx = slots.index(s)

            # LAB → needs 2 consecutive slots
            if "LAB" in subj.upper():
                if slot_idx < len(slots) - 1:
                    next_slot = slots[slot_idx + 1]
                    if (
                        next_slot not in timetable[d]
                        and "LUNCH" not in timetable[d].get(s, "")
                        and "LUNCH" not in timetable[d].get(next_slot, "")
                    ):
                        row = generate_row_for_subject(subject=subj)
                        if not violates_faculty_streak(timetable, d, slot_idx, row["Faculty"]) \
                           and not violates_faculty_streak(timetable, d, slot_idx + 1, row["Faculty"]):
                            value = f"{row['Subject']} ({row['Faculty']}, Rm {row['Room Number']})"
                            timetable[d][s] = value
                            timetable[d][next_slot] = value
                            filled += 1
                            placed = True
                            break
            else:
                # Theory subject (single slot only)
                row = generate_row_for_subject(subject=subj)
                if not violates_faculty_streak(timetable, d, slot_idx, row["Faculty"]) \
                   and not violates_theory_repeat(timetable, d, slot_idx, row["Subject"], row["Room Number"]):
                    timetable[d][s] = f"{row['Subject']} ({row['Faculty']}, Rm {row['Room Number']})"
                    filled += 1
                    placed = True
                    break
        if not placed:
            break

# Step 3: Fill remaining slots (random subjects OR FREE SLOT, but not at lunch)
for d in days:
    for s in slots:
        if s not in timetable[d]:
            if random.random() < 0.15:  # ~15% chance of free slot
                timetable[d][s] = "FREE SLOT"
                continue
            tries = 0
            while True:
                row = generate_row_for_subject()
                slot_idx = slots.index(s)

                # LAB placement check
                if "LAB" in row["Subject"].upper():
                    if slot_idx < len(slots) - 1:
                        next_slot = slots[slot_idx + 1]
                        if next_slot not in timetable[d] and "LUNCH" not in timetable[d].get(next_slot, ""):
                            value = f"{row['Subject']} ({row['Faculty']}, Rm {row['Room Number']})"
                            timetable[d][s] = value
                            timetable[d][next_slot] = value
                            break
                        else:
                            timetable[d][s] = "FREE SLOT"
                            break
                    else:
                        timetable[d][s] = "FREE SLOT"
                        break
                else:
                    if not violates_faculty_streak(timetable, d, slot_idx, row["Faculty"]) \
                       and not violates_theory_repeat(timetable, d, slot_idx, row["Subject"], row["Room Number"]):
                        timetable[d][s] = f"{row['Subject']} ({row['Faculty']}, Rm {row['Room Number']})"
                        break

                tries += 1
                if tries > 50:  # fallback
                    timetable[d][s] = "FREE SLOT"
                    break

# ==============================
# Convert to DataFrame
# ==============================
timetable_df = pd.DataFrame.from_dict(timetable, orient="index")
timetable_df = timetable_df.reindex(columns=slots)

# Clean NaNs/TBD
timetable_df = timetable_df.fillna("FREE SLOT")
timetable_df = timetable_df.replace(
    to_replace=r".*nan.*|.*TBD.*",
    value="FREE SLOT",
    regex=True
)

# ==============================
# Show nicely as a table
# ==============================
print("\n✅ Final Timetable (Excel input, labs = 2 hours, theory only single slot, lunch at 12:30 or 1:30, random free slots, no >3 faculty streaks):\n")
print(tabulate(timetable_df, headers="keys", tablefmt="grid"))

# ==============================
# Save timetable to Excel
# ==============================
OUTPUT_FILE = "Final_Timetable.xlsx"
timetable_df.to_excel(OUTPUT_FILE)
print(f"\n📂 Timetable saved to {OUTPUT_FILE}")



✅ Final Timetable (Excel input, labs = 2 hours, theory only single slot, lunch at 12:30 or 1:30, random free slots, no >3 faculty streaks):

+-----------+--------------------------------------------+-------------------------------------------------------+-----------------------------------------------------+-------------------------------------------------------+-------------------------+--------------------------------------------+-------------------------------------------------------+
|           | 9:30-10:30                                 | 10:30-11:30                                           | 11:30-12:30                                         | 12:30-1:30                                            | 1:30-2:30               | 2:30-3:30                                  | 3:30-4:30                                             |
+===========+============================================+=======================================================+========================================